In [ ]:
## import the postgres connector
import psycopg2

## INSERT THE PASSWORD

## establishing the connection
conn = psycopg2.connect(
   database="test_db", user='shoc', password='JustKeepSwimming', host='pg_container', port= '5432'
)

## Creating a cursor object using the cursor() method
cursor = conn.cursor()

cursor = conn.cursor()

## Executing an SQL function using the execute() method
cursor.execute("select * from loki_aggregated order by start_time")

## Fetch a single row using fetchone() method.
data = cursor.fetchall()
print("Pulled all of the data in loki_aggregated: ", type(data), " of length ", len(data))

## Closing the connection
conn.close()

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

## Make a dataframe from data
df = pd.DataFrame(data)

## For some reason when we pull from postgres we don't get column
##     names, so go ahead and add them back
df.columns =['container id', 'role', 'user name', 'process', 'message start', 'message stop', 'start time', 'stop time']

## I wrote a custom function to convert the cell to a datetime object
def convertStringToDateTime(cell):
    tmp = cell.to_string(header=False, index=False)
    if tmp == 'NaT':
        return None
    
    try:
        return datetime.strptime(tmp, '%Y-%m-%d %H:%M:%S.%f')
    except ValueError:
        return datetime.strptime(tmp, '%Y-%m-%d %H:%M:%S')
    
## Add another column for our df that calculates the diffence 
##     between the start and stop time
timeDelta = []
## need to merge the role and process to generate unique titles
titles = []

for i in range(0, len(df.index)):
    start = convertStringToDateTime(df.loc[i, ['start time']])
    stop = convertStringToDateTime(df.loc[i, ['stop time']])
    
    if start == None or stop == None:
        timeDelta.append(None)
    else:
        timeDelta.append((stop - start).total_seconds() / 60)
        
    titles.append(df.loc[i, ['role']].to_string(header=False, index=False)+", "+df.loc[i, ['process']].to_string(header=False, index=False))

## Now we are done add the time take column
df['titles'] = titles
df['delta'] = timeDelta

In [ ]:
import plotly.express as px
from IPython.display import HTML


# I want to order by the role/process, but only the first 50 by time
tmp = df.sort_values('start time').iloc[:100]

fig = px.timeline(tmp.sort_values('titles'), x_start="start time", x_end="stop time", y="titles", color="delta")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
HTML(fig.to_html())